# AutoGraph
TensorFlow 2.0将TensorFlow1.0的强大功能与Eager Execution模式结合起来，同时合并的核心是tf.function，它允许将Python语法的子集转换为可移植的高性能TensorFlow图。tf.function的一个很酷的新功能是AutoGraph，可以使用自然的Python语法编写图形代码。
- tf.function装饰器的作用
  - 一个函数如何被tf.function注释，会被编译成图，但是依然可以按照函数调用
  - 如果在注释的函数中，有被调用的函数，那么被调用的函数也将以图的模式运行。
  - 支持python中的所有控制语句

 ## 1. tf.function装饰器
当使用tf.function注释函数时，可以像调用任何其他函数一样调用它。它将被编译成图，这意味着可以获得更快执行，更好地在GPU或TPU上运行。

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
import numpy as np
import tensorflow as tf

In [2]:
@tf.function
def simple_nn_layer(x, y):
    return tf.nn.relu(tf.matmul(x,y))

In [3]:
x = tf.random.normal(shape=[3, 3])
y = tf.random.normal(shape=[3, 3])
simple_nn_layer(x, y)

<tf.Tensor: id=21, shape=(3, 3), dtype=float32, numpy=
array([[0.        , 0.56820613, 0.29746196],
       [0.        , 1.606781  , 3.155431  ],
       [0.        , 0.        , 0.        ]], dtype=float32)>

In [4]:
simple_nn_layer

- 注意用tf.function修饰后的simple_nn_layer函数与直接定义的simple_nn_layer的区别

In [5]:
### 不用tf.function修饰函数simple_nn_layer
def simple_nn_layer(x, y):
    return tf.nn.relu(tf.matmul(x, y))

In [6]:
simple_nn_layer(x, y)

<tf.Tensor: id=23, shape=(3, 3), dtype=float32, numpy=
array([[0.        , 0.56820613, 0.29746196],
       [0.        , 1.606781  , 3.155431  ],
       [0.        , 0.        , 0.        ]], dtype=float32)>

In [7]:
simple_nn_layer

<function __main__.simple_nn_layer(x, y)>

- #### 修饰的函数调用其他函数，这些函数也以图的模式运行

In [9]:
def linear_layer(x):
    return 2 * x + 1.0

@tf.function
def deep_net(x):
    return tf.nn.relu(linear_layer(x))

deep_net(tf.constant((1, 2.0, 3)))

<tf.Tensor: id=42, shape=(3,), dtype=float32, numpy=array([3., 5., 7.], dtype=float32)>

- #### 支持python中的所有控制语句

In [10]:
@tf.function
def sum_even(x):
    s = 0
    for iterm in x:
        if iterm % 2 != 0:
            continue
        s += iterm
    return s

result = sum_even(tf.constant([4, 7, 18, 15, 23]))
print(result)

tf.Tensor(22, shape=(), dtype=int32)


In [11]:
@tf.function
def fizzbuzz(n):
    for i in tf.range(n):
        if i % 3 == 0:
            tf.print('Fizz')
        elif i % 5 == 0:
            tf.print('Buzz')
        else:
            tf.print(i)

fizzbuzz(tf.constant(20))

Fizz
1
2
Fizz
4
Buzz
Fizz
7
8
Fizz
Buzz
11
Fizz
13
14
Fizz
16
17
Fizz
19


 ## 2.  Keras与AutoGraph
 - 可以通过注释模型的调用函数来装饰自定义Keras模型。

In [16]:
class CustomModel(tf.keras.models.Model):
    @tf.function
    def call(self, inputs):
        if tf.reduce_mean(inputs) > 0:
            return inputs
        else:
            return inputs // 2
        
model = CustomModel()
result = model(tf.constant([[1.0, 3.0, -5.0], [2.0, -7.5, 4]]))
print(result)

tf.Tensor(
[[ 0.  1. -3.]
 [ 1. -4.  2.]], shape=(2, 3), dtype=float32)


## 3.  训练函数
采用多层感知机对mnist数据集进行分类

In [1]:
import tensorflow as tf
def prepare_mnist_features_and_labels(x, y):
    x = tf.cast(x, tf.float32) / 255.0
    y = tf.cast(y, tf.int64)
    return x,y

## 加载数据--方式1：缺点是一次性读入所有的数据，消耗的内存较大
def read_mnist():
    (x, y), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
    print("mnist train data shape: ", x.shape)
    print("mnist test data shape: ", x_test.shape)
    print("mnist train data label: ", y[:30])
    ###得到迭代的数据,遍历整个数据集
    data = tf.data.Dataset.from_tensor_slices((x, y)) # 对输入的tensor以第一个维度切分数据，例如tensor的shape(200,28,28,3),切分200个shape为(28,28,3)的tensor
    data = data.map(prepare_mnist_features_and_labels)
    ### 筛选一部分数据作为训练集
    # data = data.take(20000).shuffle(20000).batch(100)
    data = data.shuffle(60000).batch(100)
    return data

In [2]:
### 定义多层感知机模型
model = tf.keras.Sequential()
model.add(tf.keras.layers.Reshape(target_shape=(28 * 28,), input_shape=(28, 28)))
model.add(tf.keras.layers.Dense(200, activation="relu"))
model.add(tf.keras.layers.Dense(100, activation="relu"))
model.add(tf.keras.layers.Dense(100, activation="relu"))
model.add(tf.keras.layers.Dense(10))
model.build()
## 定义优化器
optimizer = tf.keras.optimizers.Adam()
### 定义loss
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
### 定义准确率
accuracy = tf.keras.metrics.SparseCategoricalAccuracy()
### 定义梯度优化
def one_step_gradient(model, optimizer, x, y):
    with tf.GradientTape() as tape:
        logits = model(x)
        lossing = loss(y, logits)
    ## 梯度
    grads = tape.gradient(lossing, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    ## predict
    predict = tf.nn.softmax(logits) ### 这个softmax到底需要不需要
    accuracy(y, predict)
    return lossing
### 修饰训练函数
@tf.function
def training(model, optimizer):
    ## 加载数据集
    train_data = read_mnist()
    epoches = 10
    training_loss = 0.0
    train_accuracy = 0.0
    for epoch in range(epoches):
        step = 0
        for batch_x, batch_y in train_data:
            step += 1
            training_loss = one_step_gradient(model, optimizer, batch_x, batch_y)
            if step % 100 == 0:
                tf.print("epoch: ", epoch + 1, "step: ", step, "training loss: ", training_loss, 
                         "training accuracy: ", accuracy.result())
    return step, training_loss, train_accuracy

In [3]:
step, training_loss, train_accuracy = training(model, optimizer)
print("final step:{} final loss:{} accuracy:{}".format(step, training_loss, accuracy.result()))

mnist train data shape:  (60000, 28, 28)
mnist test data shape:  (10000, 28, 28)
mnist train data label:  [5 0 4 1 9 2 1 3 1 4 3 5 3 6 1 7 2 8 6 9 4 0 9 1 1 2 4 3 2 7]
mnist train data shape:  (60000, 28, 28)
mnist test data shape:  (10000, 28, 28)
mnist train data label:  [5 0 4 1 9 2 1 3 1 4 3 5 3 6 1 7 2 8 6 9 4 0 9 1 1 2 4 3 2 7]
epoch:  1 step:  10 training loss:  1.63552809 training accuracy:  0.452
epoch:  1 step:  20 training loss:  0.769016325 training accuracy:  0.5855
epoch:  1 step:  30 training loss:  0.452850223 training accuracy:  0.664666653
epoch:  1 step:  40 training loss:  0.367639244 training accuracy:  0.70975
epoch:  1 step:  50 training loss:  0.565599203 training accuracy:  0.743
epoch:  1 step:  60 training loss:  0.382287413 training accuracy:  0.767166674
epoch:  1 step:  70 training loss:  0.412267685 training accuracy:  0.785428584
epoch:  1 step:  80 training loss:  0.424208283 training accuracy:  0.80025
epoch:  1 step:  90 training loss:  0.421686709 tr

epoch:  2 step:  400 training loss:  0.0582469888 training accuracy:  0.94059
epoch:  2 step:  410 training loss:  0.0396924764 training accuracy:  0.940891087
epoch:  2 step:  420 training loss:  0.0955961645 training accuracy:  0.941264689
epoch:  2 step:  430 training loss:  0.0255029611 training accuracy:  0.941514552
epoch:  2 step:  440 training loss:  0.0563107878 training accuracy:  0.941894233
epoch:  2 step:  450 training loss:  0.104376018 training accuracy:  0.942133307
epoch:  2 step:  460 training loss:  0.040781159 training accuracy:  0.942415118
epoch:  2 step:  470 training loss:  0.0387414731 training accuracy:  0.942719638
epoch:  2 step:  480 training loss:  0.157482266 training accuracy:  0.94299072
epoch:  2 step:  490 training loss:  0.103141077 training accuracy:  0.943211
epoch:  2 step:  500 training loss:  0.198927 training accuracy:  0.943445444
epoch:  2 step:  510 training loss:  0.153759256 training accuracy:  0.943720698
epoch:  2 step:  520 training los

epoch:  4 step:  220 training loss:  0.0633514524 training accuracy:  0.959792078
epoch:  4 step:  230 training loss:  0.0257479195 training accuracy:  0.959896564
epoch:  4 step:  240 training loss:  0.0114299925 training accuracy:  0.960049033
epoch:  4 step:  250 training loss:  0.0468564704 training accuracy:  0.960161
epoch:  4 step:  260 training loss:  0.0104713961 training accuracy:  0.960296094
epoch:  4 step:  270 training loss:  0.0616436526 training accuracy:  0.960405827
epoch:  4 step:  280 training loss:  0.0112006404 training accuracy:  0.960485578
epoch:  4 step:  290 training loss:  0.0444429927 training accuracy:  0.960626781
epoch:  4 step:  300 training loss:  0.0933836922 training accuracy:  0.960728586
epoch:  4 step:  310 training loss:  0.0266094953 training accuracy:  0.960843623
epoch:  4 step:  320 training loss:  0.0500821732 training accuracy:  0.960924506
epoch:  4 step:  330 training loss:  0.00926425308 training accuracy:  0.961009383
epoch:  4 step:  3

epoch:  6 step:  30 training loss:  0.0550001971 training accuracy:  0.968280554
epoch:  6 step:  40 training loss:  0.0606565103 training accuracy:  0.968335509
epoch:  6 step:  50 training loss:  0.025787726 training accuracy:  0.968406558
epoch:  6 step:  60 training loss:  0.0134201869 training accuracy:  0.96847713
epoch:  6 step:  70 training loss:  0.0187880546 training accuracy:  0.968547225
epoch:  6 step:  80 training loss:  0.0190022159 training accuracy:  0.968613625
epoch:  6 step:  90 training loss:  0.00367343449 training accuracy:  0.968692541
epoch:  6 step:  100 training loss:  0.0126949428 training accuracy:  0.968771
epoch:  6 step:  110 training loss:  0.0312536955 training accuracy:  0.968848884
epoch:  6 step:  120 training loss:  0.00953768659 training accuracy:  0.968900621
epoch:  6 step:  130 training loss:  0.0321978964 training accuracy:  0.96895206
epoch:  6 step:  140 training loss:  0.0199158918 training accuracy:  0.969025493
epoch:  6 step:  150 traini

epoch:  7 step:  440 training loss:  0.011106045 training accuracy:  0.973928213
epoch:  7 step:  450 training loss:  0.00730133476 training accuracy:  0.97396791
epoch:  7 step:  460 training loss:  0.00946734473 training accuracy:  0.974007368
epoch:  7 step:  470 training loss:  0.00944985729 training accuracy:  0.974056482
epoch:  7 step:  480 training loss:  0.00998330861 training accuracy:  0.974105418
epoch:  7 step:  490 training loss:  0.0774801522 training accuracy:  0.974146724
epoch:  7 step:  500 training loss:  0.00148397451 training accuracy:  0.974192679
epoch:  7 step:  510 training loss:  0.0232252628 training accuracy:  0.97422874
epoch:  7 step:  520 training loss:  0.00733581651 training accuracy:  0.974245131
epoch:  7 step:  530 training loss:  0.0403766967 training accuracy:  0.974273622
epoch:  7 step:  540 training loss:  0.00806467608 training accuracy:  0.974314034
epoch:  7 step:  550 training loss:  0.00767880306 training accuracy:  0.974356651
epoch:  7 s

epoch:  9 step:  250 training loss:  0.0529522076 training accuracy:  0.977459431
epoch:  9 step:  260 training loss:  0.0215214156 training accuracy:  0.977496
epoch:  9 step:  270 training loss:  0.00275702844 training accuracy:  0.977522671
epoch:  9 step:  280 training loss:  0.012729777 training accuracy:  0.977555096
epoch:  9 step:  290 training loss:  0.00263827224 training accuracy:  0.977585435
epoch:  9 step:  300 training loss:  0.000374119874 training accuracy:  0.97761178
epoch:  9 step:  310 training loss:  0.0192162376 training accuracy:  0.977649689
epoch:  9 step:  320 training loss:  0.0025310833 training accuracy:  0.977683604
epoch:  9 step:  330 training loss:  0.0779746622 training accuracy:  0.977715373
epoch:  9 step:  340 training loss:  0.00616634591 training accuracy:  0.97774905
epoch:  9 step:  350 training loss:  0.00641707657 training accuracy:  0.977784455
epoch:  9 step:  360 training loss:  0.0280789118 training accuracy:  0.977808118
epoch:  9 step: 

- #### 并不是所有的都支持AutoGraph模式

In [4]:
import tensorflow as tf
## 加载数据--方式2：批量读入数据
def read_data():
    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
    print("mnist traing data shape: ", x_train.shape)
    print("mnist test data shape: ", x_test.shape)
    x_train = tf.cast(x_train, tf.float32) / 255.0
    y_train = tf.cast(y_train, tf.int64)
    def get_batch():
        batch_size = 100
        batch_num = int(x_train.shape[0] // batch_size)
        for i in range(batch_num):
            batch_x = x_train[i * batch_size: (i+1) * batch_size, :, :]
            batch_y = y_train[i * batch_size: (i+1) * batch_size]
            yield batch_x, batch_y
    data = tf.data.Dataset.from_generator(get_batch, (tf.float32, tf.int64)) ## 
    return data

In [9]:
### 定义多层感知机模型
model = tf.keras.Sequential()
model.add(tf.keras.layers.Reshape(target_shape=(28 * 28,), input_shape=(28, 28)))
model.add(tf.keras.layers.Dense(200, activation="relu"))
model.add(tf.keras.layers.Dense(100, activation="relu"))
model.add(tf.keras.layers.Dense(100, activation="relu"))
model.add(tf.keras.layers.Dense(10))
model.build()
## 定义优化器
optimizer = tf.keras.optimizers.Adam()
### 定义loss
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
### 定义准确率
accuracy = tf.keras.metrics.SparseCategoricalAccuracy()
### 定义梯度优化
def one_step_gradient(model, optimizer, x, y):
    with tf.GradientTape() as tape:
        logits = model(x)
        lossing = loss(y, logits)
    ## 梯度
    grads = tape.gradient(lossing, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    ## predict
    predict = tf.nn.softmax(logits) ### 这个softmax到底需要不需要
    accuracy(y, predict)
    return lossing

def training(model, optimizer):
    ## 加载数据集
    # train_data = read_mnist()
    train_data = read_data()
    epoches = 10
    training_loss = 0.0
    train_accuracy = 0.0
    for epoch in range(epoches):
        step = 0
        for batch_x, batch_y in train_data:
            step += 1
            training_loss = one_step_gradient(model, optimizer, batch_x, batch_y)
            if step % 100 == 0:
                tf.print("epoch: ",epoch + 1, "step: ", step, "training loss: ", training_loss, 
                         "training accuracy: ",accuracy.result())
    return step, training_loss, train_accuracy

In [10]:
step, training_loss, train_accuracy = training(model, optimizer)
print("final step:{} final loss:{} accuracy:{}".format(step, training_loss, accuracy.result()))

mnist traing data shape:  (60000, 28, 28)
mnist test data shape:  (10000, 28, 28)
epoch:  1 step:  100 training loss:  0.193819642 training accuracy:  0.7991
epoch:  1 step:  200 training loss:  0.206223711 training accuracy:  0.854
epoch:  1 step:  300 training loss:  0.321686298 training accuracy:  0.88076669
epoch:  1 step:  400 training loss:  0.287481755 training accuracy:  0.89515
epoch:  1 step:  500 training loss:  0.214030355 training accuracy:  0.90436
epoch:  1 step:  600 training loss:  0.200359613 training accuracy:  0.913683355
epoch:  2 step:  100 training loss:  0.0579241738 training accuracy:  0.9206
epoch:  2 step:  200 training loss:  0.0396165289 training accuracy:  0.92605
epoch:  2 step:  300 training loss:  0.126213253 training accuracy:  0.93032223
epoch:  2 step:  400 training loss:  0.131759942 training accuracy:  0.93414
epoch:  2 step:  500 training loss:  0.111321948 training accuracy:  0.9371
epoch:  2 step:  600 training loss:  0.187709063 training accura